# Análise de Sentimentos

Treinando a machine learning, utilizando o dataset recebido por email. (Faça o download e coloque a pasta dentro do repositorio)

Este notebook faz o treinando de machine learning de um modelo de análise de sentimentos, utilizando o dataset do Buscapé fornecido (confidencial), de forma que:
- Carrega e estrutura os dados do dataset em dicionário
- Exibe estatísticas acerca da do dataset
- Apresenta comentários acerca dos resultados obtidos

## Configurações Iniciais

Primeiramente, fazer o ajuste no path do Python para a raíz do projeto:

In [ ]:
import os
os.chdir('..')

--------------------------------------

## Processamento do Dataset

Carregar a bliblioteca de normalização de texto e de dump de dados:

In [ ]:
from nlputils.lexical.normalizer import Normalizer
import pickle

Instanciar um objeto para fazer as normalizações necessárias:

In [ ]:
normalizer = Normalizer()

Implementar uma rotina de pré-processamento a ser aplicada em todo o dataset antes de construir o modelo e uma para auxiliar o carregamento dos dados, evitando replicação de código.

A função ```preprocessing``` recebe um texto (string) de entrada e o deixa normalizado, conforme procedimentos de nível lexical, e retorna o próprio texto (string) mais compacto.

In [ ]:
def preprocessing(text):
    text = normalizer.to_lowercase(text)
    text = normalizer.remove_ponctuations(text)
    tokens = normalizer.tokenize_words(text)
    tokens = normalizer.remove_stopwords(tokens)
    return ' '.join(tokens)

A função ```all_files``` recebe o caminho do dataset a ser carregado e gera o caminho de todos os arquivos XML a serem lidos. O caminho de entrada deve terminal com ```/```.

In [3]:
def all_files(dataset_path):
    for directory in os.listdir(dataset_path):
        if directory != '.DS_Store':
            for stars in os.listdir(dataset_path + directory):
                for file in os.listdir(dataset_path + directory + '/' + stars):
                    if file.endswith('.xml'):
                        yield '{}{}/{}/{}'.format(dataset_path, directory, stars, file)

A função ```dumpvar``` salva em dados binário todo o valor de uma variável na memória. Basta fornecer o ponteiro do identificador da variável e o caminho onde será salvo o dump.

In [ ]:
def dumpvar(var, path):
    path_itens = path.split('/')
    path_dir = '/'.join(path_itens[:-1]) + '/'
    
    if not os.path.isdir(path_dir):
        os.makedirs(path_dir)
    
    with open(path, 'wb') as fp:
        pickle.dump(var, fp)

A função ```loadvar``` retorna o valor de uma dump de memória salvo em disco. Basta o caminho onde está salvo o dump e atribuir em uma variável.

In [ ]:
def loadvar(path):
    with open(path, 'rb') as fp:
        pickle.dump(var, fp)

---------------------------------

## Carregando o Dataset

Importar as bibliotecas utilizadas para fazer o parse do XML e criar a estrutura.

In [1]:
import xmltodict
import pandas as pd

Carregando os arquivos XML, exibindo estatísticas sobre eles e criando a estrutura que será utilizada no modelo de Machine Learning.

Aqui, o dataset está sendo repartido ao meio para criar o conjunto de treinamento e o de teste.

In [4]:
dataset = {'trainset':[], 'polarity':[], 'review':[]}
all_files_list = list(all_files('data/trainset/'))
partition_size = len(all_files) // 2

for idx, file in enumerate(all_files_list):
    filename = file.split('/')[-1]
    with open(file, 'r') as text_file:
        data = text_file.read()
        try:
            dict_data = xmltodict.parse(data)
            dataset['trainset'].append('train') if idx > partition_size else dataset['trainset'].append('test')
            polarity = float(dict_data['review']['stars']['@value'])
            dataset['polarity'].append(polarity)
            dataset['review'].append(str(dict_data['review']['opinion']))
        except:
            print('Arquivo {} está com xml mal formatado'.format(file))

Arquivo trainset/Tablet/4.0/0_335616.xml está com xml mal formatado
Arquivo trainset/Umidificador/4.0/7_393334.xml está com xml mal formatado
Arquivo trainset/HD/5.0/0_336852.xml está com xml mal formatado
Arquivo trainset/Notebook/3.0/0_343109.xml está com xml mal formatado
Arquivo trainset/Celular e Smartphone/4.0/2_42055.xml está com xml mal formatado


In [6]:
dataframe = pd.DataFrame(data=dataset)
# dataframe.head()

,trainset,polarity,review
0,test,0.0,eu amei o violao e eu quero muito avaliar\n\nO...
1,test,4.0,É um ótimo violão tanto para usar em casa quan...
2,test,4.0,"Um produto de excelente, qualidade e leitura"
3,test,4.0,"Um produto de excelente, qualidade e leitura\n..."
4,test,4.0,"Toquei e achei o Maximo, toque e voce tambem v..."


Agora vamos ler todos os textos e adicionar um novo campo na dataframe que irá ter a sentença sem pontuações e stopwords

In [13]:
dataframe['normalized_review'] = dataframe['review'].apply(preprocessing)
# dataframe.head()

,trainset,polarity,review,normalized_review
0,test,0.0,eu amei o violao e eu quero muito avaliar\n\nO...,amei violao quero avaliar gostei cor
1,test,4.0,É um ótimo violão tanto para usar em casa quan...,é ótimo violão tanto usar casa quanto profissi...
2,test,4.0,"Um produto de excelente, qualidade e leitura",produto excelente qualidade leitura
3,test,4.0,"Um produto de excelente, qualidade e leitura\n...",produto excelente qualidade leitura gostei alg...
4,test,4.0,"Toquei e achei o Maximo, toque e voce tambem v...",toquei achei maximo toque voce tambem vai sent...


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

train_reviews = dataframe[dataframe['trainset'] == 'train']['normalized_review'].values.tolist()
train_classes = dataframe[dataframe['trainset'] == 'train']['polarity'].values.tolist()
test_reviews = dataframe[dataframe['trainset'] == 'test']['normalized_review'].values.tolist()
test_classes = dataframe[dataframe['trainset'] == 'test']['polarity'].values.tolist()

transformer = TfidfVectorizer()
transformer.fit(train_reviews)
X = transformer.transform(train_reviews)
X_test = transformer.transform(test_reviews)

Treinando 

In [15]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [122]:
classifier = SVC()
classifier.fit(X, train_classes)

/home/bruno/Projetos/.virtualenvs/pln-ven/lib/python3.5/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [124]:
accuracy_score(test_classes, classifier.predict(X_test))

0.37370211242391693

In [125]:
classifier_lr = LogisticRegression()
classifier_lr.fit(X, train_classes)

/home/bruno/Projetos/.virtualenvs/pln-ven/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bruno/Projetos/.virtualenvs/pln-ven/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [126]:
accuracy_score(test_classes, classifier_lr.predict(X_test))

0.507906671440506

In [135]:
sentence = "Esse filme é fraco"
preprocessed_sentence = preprocessing(sentence)
print(preprocessed_sentence)
instance = transformer.transform([preprocessing(sentence)])
print(instance)
classifier_lr.predict(instance)

filme é fraco
  (0, 21978)	0.6504526192491843
  (0, 21348)	0.7595468320728326


array([3.])

In [136]:
sentence = "Esse filme é fraco"
preprocessed_sentence = preprocessing(sentence)
print(preprocessed_sentence)
instance = transformer.transform([preprocessing(sentence)])
print(instance)
classifier.predict(instance)

filme é fraco
  (0, 21978)	0.6504526192491843
  (0, 21348)	0.7595468320728326


array([5.])

In [137]:
import pickle

